# sp_BlitzFirst from Brent Ozar

You can download the First Responder Kit from https://www.brentozar.com/first-aid/ It will provide you with an excellent set of resources for free including sp_Blitz.

You can either run the Install-All-Scripts.sql script to install all of the stored procedures or the individual stored procedures using the T-SQL files. It can go i nthe master database or in a DBA database, it is up to you. 

With the stored procedures installed you can then investigate the instance that you are interested in. Brent has written about how to start here https://www.brentozar.com/archive/2017/10/free-sql-server-performance-check/

## Run sp_blitzFirst with basic parameters

Start with wait stats, which tell you what the SQL Server has been waiting on since the service was last started, or since someone cleared wait stats with DBCC SQLPERF, whichever is more recent.


In [1]:
	EXEC sp_BlitzFirst @SinceStartup = 1;

# Common sp_BlitzFirst Parameters

@ExpertMode = 1 – doesn’t do more in-depth checks, but DOES return a whole lot more data, including sections on wait stats, file stats, and Perfmon counters. Calls sp_BlitzWho at the start & end too so you can see which queries were running at those times. @SinceStartup = 1 – displays wait stats, file stats, and Perfmon counters since the SQL Server started up. @Seconds = 5 – by default, we take a 5-second sample of activity, but you can take a longer sample if you want to run a load test or demo.

In [2]:
EXEC sp_BlitzFirst @ExpertMode = 1, @Seconds = 5

# Put the results into a table

You can also output the results and place them into a database table - this would be good to run as an agent job

**DON't use the tempdb though!!**

In [11]:
EXEC sp_BlitzFirst 
  @OutputDatabaseName = 'tempdb', 
  @OutputSchemaName = 'dbo', 
  @OutputTableName = 'BlitzFirst',
  @OutputTableNameFileStats = 'BlitzFirst_FileStats',
  @OutputTableNamePerfmonStats = 'BlitzFirst_PerfmonStats',
  @OutputTableNameWaitStats = 'BlitzFirst_WaitStats',
  @OutputTableNameBlitzCache = 'BlitzCache'

Once the query above has been run more than once (I think an Agent job would do this for you automatically on a schedule!!)

You can query the results

See the top 5 Waits for example

In [14]:
USE TempDb  --- DON'T Use TempDB
GO
DECLARE @TOPWAITS NVARCHAR(500) = ''
SELECT @TOPWAITS += QUOTENAME(wait_type)+ ','
FROM (
    SELECT TOP 5 WAIT_TYPE
    FROM dbo.BlitzFirst_WaitStats_Deltas
    GROUP BY wait_type
    ORDER BY SUM([wait_time_minutes_delta]) DESC
)AS WAITS
SET @TOPWAITS = LEFT(@TOPWAITS, LEN(@TOPWAITS)-1)
DECLARE @WAITQUERY NVARCHAR(MAX)
SET @WAITQUERY = '
SELECT * FROM (
    SELECT CheckDate
        , wait_type
        , wait_time_minutes_per_minute
    FROM BlitzFirst_WaitStats_Deltas WAITS
    WHERE WAIT_TYPE IN
        (SELECT top 3 [wait_type]
            FROM dbo.BlitzFirst_WaitStats_Deltas
            GROUP BY wait_type
            ORDER BY SUM([wait_time_minutes_delta]) DESC)
) WTS

PIVOT
    (SUM(WAIT_TIME_MINUTES_PER_MINUTE)
        FOR WAIT_TYPE IN 
        ('+@TOPWAITS+')
    ) AS WAITGRAPH;
'
EXECUTE SP_EXECUTESQL @WAITQUERY

Commands completed successfully.

Warning: The join order has been enforced because a local join hint is used.

Warning: The join order has been enforced because a local join hint is used.

(2 rows affected)

Total execution time: 00:00:00.068

CheckDate,CXPACKET,ASYNC_IO_COMPLETION,BACKUPIO,LATCH_EX,PAGEIOLATCH_SH
2020-01-30 13:15:34.6011824 +01:00,2.00326582278481012,3.01534177215189873,2.89544303797468354,NULL,NULL
2020-01-30 13:37:24.9512226 +01:00,5.74637862595419847,2.99602977099236641,2.72722824427480916,NULL,NULL


Top Resource Instensive Queries

In [ ]:
USE TempDb  --- DON'T Use TempDB
GO
SELECT TOP 100 CheckDate
    , DatabaseName
    , QueryType
    , Warnings
    , left(QueryText,100) as [QueryText100]
    , [Query Hash More Info]
FROM dbo.BlitzCache
ORDER BY CHECKDATE DESC

## Getting historical data

You can use the stored procedure to understand what was happening at that moment in history as well

In [ ]:
EXEC sp_BlitzFirst @AsOf = '2020-01-30 12:03', @OutputDatabaseName = 'TempDb', @OutputSchemaName = 'dbo', @OutputTableName = 'BlitzFirst'